In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

In [2]:
spark = SparkSession.builder.appName("spark-sql-example-app").getOrCreate()

25/02/04 01:34:10 WARN Utils: Your hostname, codespaces-10f702 resolves to a loopback address: 127.0.0.1; using 10.0.1.11 instead (on interface eth0)
25/02/04 01:34:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 01:34:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
csv_file = "./datasets/flights/departuredelays.csv"
df = (
    spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csv_file)
)
df.createOrReplaceTempView("us_delay_flights_tbl")

In [9]:
df.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [10]:
# 비행거리가 1000마일 이상인 상위 10개 항공편
spark.sql(
    """
        SELECT 
      DISTINCT   distance
               , origin
               , destination
          FROM   us_delay_flights_tbl
         WHERE   distance > 1000
         ORDER
            BY   distance DESC            
    ;
    """
).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   JFK|        HNL|
|    4330|   HNL|        JFK|
|    4312|   HNL|        EWR|
|    4312|   EWR|        HNL|
|    4186|   IAD|        HNL|
|    4186|   HNL|        IAD|
|    3912|   ATL|        HNL|
|    3912|   HNL|        ATL|
|    3687|   HNL|        ORD|
|    3687|   ORD|        HNL|
+--------+------+-----------+
only showing top 10 rows



In [11]:
# 샌프란시스코(SFO)와 시카고(ORD) 간 2시간 이상 지연이 있었던 상위 10개 항공편
spark.sql(
    """
        SELECT   date
               , delay
               , origin
               , destination
          FROM   us_delay_flights_tbl
         WHERE   delay > 120
           AND   origin = 'SFO'
           AND   destination = 'ORD'
         ORDER
            BY   delay DESC
    """
).show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [12]:
spark.sql(
    """
        SELECT   delay
               , origin
               , destination
               , CASE
                    WHEN delay > 360 THEN 'Very Long Delays'
                    WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
                    WHEN delay >= 60 AND delay <= 120 THEN 'Short Delays'
                    WHEN delay >= 0 AND delay < 60 THEN 'Tolarable Delays'
                    WHEN delay = 0 THEN 'No Delays'
                    ELSE 'Early'
                 END AS flight_delays
          FROM   us_delay_flights_tbl
         ORDER
            BY   origin
               , delay DESC
    
    """
).show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [17]:
(
    df
    .select("distance", "origin", "destination")
    .distinct()
    .where(col("distance") > 1000)
    .orderBy(desc("distance"))
).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   JFK|        HNL|
|    4330|   HNL|        JFK|
|    4312|   HNL|        EWR|
|    4312|   EWR|        HNL|
|    4186|   IAD|        HNL|
|    4186|   HNL|        IAD|
|    3912|   ATL|        HNL|
|    3912|   HNL|        ATL|
|    3687|   HNL|        ORD|
|    3687|   ORD|        HNL|
+--------+------+-----------+
only showing top 10 rows



In [4]:
spark.sql("CREATE DATABASE learn_spark_db;")
spark.sql("USE learn_spark_db;")

25/02/04 01:34:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


DataFrame[]

In [ ]:
spark.sql(
    """
        CREATE TABLE managed_us_delay_flights_tbl (
              date          STRING
            , delay         INT
            , distance      INT
            , origin        STRING
            , destination   STRING
        )
    
    """
)

csv_file = "./datasets/flights/departuredelays.csv"
schema = """
      date          STRING
    , delay         INT
    , distance      INT
    , origin        STRING
    , destination   STRING
"""

flights_df = spark.read_csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

In [5]:
spark.sql(
    """
        CREATE TABLE us_delay_flights_tbl (
              date          STRING
            , delay         INT
            , distance      INT
            , origin        STRING
            , destination   STRING
        )
    
        USING csv OPTIONS(PATH './datasets/flights/departuredelays.csv')
    """
)

25/02/04 01:40:29 WARN HadoopFSUtils: The directory file:/workspaces/scratchs/platforms/apache-spark/learning-spark-v2/spark-warehouse/learn_spark_db.db/datasets/flights/departuredelays.csv was not found. Was it deleted very recently?


DataFrame[]

In [11]:
df_sfo = spark.sql(
    """
        SELECT   date
               , delay
               , origin
               , destination
          FROM   us_delay_flights_tbl
         WHERE   origin = 'SFO'       
    """
)
df_jfk = spark.sql(
    """
        SELECT   date
               , delay
               , origin
               , destination
          FROM   us_delay_flights_tbl
         WHERE   origin = 'JFK'       
    """
)

df_sfo.createOrReplaceGlobalTempView("us_origin_airport_sfo_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_jfk_tmp_view")

In [13]:
spark.sql("""
    SELECT   *
      FROM   us_origin_airport_jfk_tmp_view          
""").show()

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
|1011540|   -4|   JFK|        DFW|
|1011705|    5|   JFK|        SAN|
|1011530|   -3|   JFK|        SFO|
|1011630|   -3|   JFK|        SJU|
|1011345|    2|   JFK|        LAX|
|1011545|   -3|   JFK|        LAX|
|1011510|   -1|   JFK|        MIA|
|1011745|    7|   JFK|        SFO|
|1011250|    3|   JFK|        BOS|
|1011645|  142|   JFK|        LAX|
|1012135|   -2|   JFK|        LAX|
|1011715|   18|   JFK|        ORD|
|1011615|   25|   JFK|        IAH|
|1011850|   -2|   JFK|        SEA|
|1011725|   -5|   JFK|        BOS|
+-------+-----+------+-----------+
only showing top 20 rows



In [14]:
spark.catalog.dropGlobalTempView("us_origin_airport_sfo_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_jfk_tmp_view")

True

In [15]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/workspaces/scratchs/platforms/apache-spark/learning-spark-v2/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/workspaces/scratchs/platforms/apache-spark/learning-spark-v2/spark-warehouse/learn_spark_db.db')]

In [16]:
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
spark.catalog.listColumns("learn_spark_db.us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]